In [3]:
# Import Required Libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

# Compute Expected Genetic Relatedness under the Factor Model

To estimate broad-sense heritability under the Factor model ($H^2_{\text{Factor}}$), we convert SNP-based variance components into expected twin-based heritability using Falconer’s formula:

$$
H^2 = 2(r_{\text{MZ}} - r_{\text{DZ}})
$$

**Step 1**: MZ and DZ Twin Correlations

Because monozygotic twins are genetically identical, we assume that all modeled genetic variance is fully shared within each pair. Hence, under the Factor model:

$$
r_{\text{MZ}} = h^2_{\text{het}} + h^2_{\text{hom}}
$$

For dizygotic twins, the expected genetic correlation depends on allele sharing probabilities, which vary with the minor allele frequency (MAF).

Step 2: Estimating MAF-dependent Relatedness ($\gamma$)

In the FACTOR model, the genetic relatedness of variance components (hetero and homozygous and their interaction) is no longer linear in MAF. Thus, we empirically estimate genetic relatedness ($\gamma$) as follows:

1.	For each MAF in the range 0.01–0.5, we simulate genotypes for DZ twin pairs.
2.	We compute expected genetic relatedness for heterozygous, homozyous, and their interaction ($\gamma_{\text{het}}, \gamma_{\text{hom}}$ and $\gamma_{\text{het} \times \text{hom}}$).
3.	We then average these values, weighting them by the MAF distribution observed in the UK Biobank.

The expected DZ twin correlation is calculated as:
$$
r_{\text{DZ}} = E[\gamma_{\text{het}}] \cdot \sigma^2_{\text{het}} + E[\gamma_{\text{hom}}] \cdot \sigma^2_{\text{hom}} + 2,E[\gamma_{\text{het} \times \text{hom}}] \cdot \sigma_{\text{het} \times \text{hom}}
$$

Step 3: Applying Falconer’s Formula

Finally, we use the model-specific $r_{\text{MZ}}$ and $r_{\text{DZ}}$ correlations to compute the implied twin-based heritability under the Factor model.
This approach allows direct comparison with heritability estimates reported in empirical twin studies.


## Make MAF-dependent DZ twin genetic relatedness table via simualtion

In [4]:
# Genotype generation functions
def make_dz_genotype_for_sinlge_locus(n_pairs: int, f: float, seed: int):
    rng = np.random.default_rng(seed)

    # Parental haplotypes
    father = rng.binomial(1, f, size=(n_pairs, 2))
    mother = rng.binomial(1, f, size=(n_pairs, 2))
    
    # Gamete sampling
    g1 = father[np.arange(n_pairs), rng.integers(0, 2, size=n_pairs)] + \
         mother[np.arange(n_pairs), rng.integers(0, 2, size=n_pairs)]
    g2 = father[np.arange(n_pairs), rng.integers(0, 2, size=n_pairs)] + \
         mother[np.arange(n_pairs), rng.integers(0, 2, size=n_pairs)]
    
    return g1, g2

def encoding_Factor(g: np.ndarray, f: float):
     U = np.stack([g == 1, g == 2], axis=1).astype(float)
     
     # Standardize
     p1, p2 = 2*f*(1-f), f**2
     mu = np.array([p1, p2])
     a = p1*(1-p1)
     b = -p1*p2
     c = p2*(1-p2)
     det = a*c - b**2

     W = np.array([[1/np.sqrt(a), 0],
                    [p1*p2/np.sqrt(a*det), np.sqrt(a/det)]])

     U_std = (U - mu) @ W.T

     return U, U_std

def dz_cov(n_pairs: int, maf: float, seed: int):
    g1, g2 = make_dz_genotype_for_sinlge_locus(n_pairs, maf, seed)
    
    # Factor model
    _, X1_std = encoding_Factor(g1, maf)
    _, X2_std = encoding_Factor(g2, maf)
    cov_Factor = X1_std.T @ X2_std / n_pairs
    
    return cov_Factor

In [41]:
# Iterative simulation
n_pairs = 1_000_000
df_dz_gr = pd.DataFrame(columns=["MAF", "het", "hom", "hetxhom"])

for f in tqdm(np.arange(0.01, 0.5 + 0.01, 0.01)):
    for _ in range(100):
        gamma_Factor = dz_cov(n_pairs, f, np.random.randint(0, 10000))
        df_dz_gr = pd.concat([df_dz_gr, pd.DataFrame({
            "MAF": f,
            "het": [gamma_Factor[0, 0]],
            "hom": [gamma_Factor[1, 1]],
            "hetxhom": [gamma_Factor[0, 1]]
        })])


  0%|          | 0/50 [00:00<?, ?it/s]/var/folders/2b/0852f9994h3dls0xx_wwhjxc0000gn/T/ipykernel_5507/2066000487.py:8: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_dz_gr = pd.concat([df_dz_gr, pd.DataFrame({
100%|██████████| 50/50 [09:24<00:00, 11.28s/it]


In [42]:
df_dz_gr

,MAF,het,hom,hetxhom
0,0.01,0.496516,0.260030,0.033857
0,0.01,0.492071,0.300021,0.031581
0,0.01,0.496445,0.240019,0.040725
0,0.01,0.495096,0.110052,0.041079
0,0.01,0.495525,0.250021,0.030261
...,...,...,...,...
0,0.50,0.251170,0.499570,0.002113
0,0.50,0.250134,0.500020,0.000853
0,0.50,0.248254,0.498558,0.000430
0,0.50,0.250044,0.499920,0.000595


In [44]:
df_dz_gr.groupby("MAF").mean().head()

,het,hom,hetxhom
MAF,,,
0.01,0.494910,0.246525,0.034816
0.02,0.489042,0.264571,0.049269
0.03,0.484580,0.264148,0.060051
0.04,0.479331,0.272077,0.069089
0.05,0.473872,0.277898,0.076618


## Load UKB MAF distribution

In [45]:
df_mafs = pd.read_csv(
    "../../data/UKB.merged.afreq",
    sep='\t'
)
df_mafs

,#CHROM,ID,REF,ALT,ALT_FREQS,OBS_CT
0,1,rs12238997,A,G,0.094106,576212
1,1,rs200531508,G,A,0.026765,570522
2,1,rs12071806,T,G,0.010810,580206
3,1,rs144155419,G,A,0.010860,616284
4,1,rs116587930,G,A,0.035947,582188
...,...,...,...,...,...,...
5751796,22,rs62240034,T,G,0.026119,576816
5751797,22,rs62240035,C,G,0.026119,576816
5751798,22,rs368226325,A,G,0.042218,602562
5751799,22,rs374914422,C,T,0.015130,601126


In [46]:
# MAF bins 
bins = np.arange(0.005, 0.505, 0.01)  # [0.005, 0.015, 0.025, ..., 0.495]
bin_means = np.arange(0.01, 0.51, 0.01)  # [0.01, 0.02, 0.03, ..., 0.50]

# MAF 데이터를 bins로 분류
df_mafs['MAF_bin'] = pd.cut(df_mafs['ALT_FREQS'], bins=bins, include_lowest=True)

# Count와 Proportion 계산
maf_stats = df_mafs['MAF_bin'].value_counts().sort_index()
maf_stats_prop = df_mafs['MAF_bin'].value_counts(normalize=True).sort_index()

# 결과 DataFrame 생성
result = pd.DataFrame({
    'MAF_bin': maf_stats.index,
    'Count': maf_stats.values,
    'Proportion': maf_stats_prop.values,
    'MAF': bin_means[:len(maf_stats)]
})

In [33]:
result.head()

,MAF_bin,Count,Proportion,MAF
0,"(0.004, 0.015]",420880,0.073546,0.01
1,"(0.015, 0.025]",561792,0.098169,0.02
2,"(0.025, 0.035]",370703,0.064778,0.03
3,"(0.035, 0.045]",279448,0.048831,0.04
4,"(0.045, 0.055]",230913,0.040350,0.05


## Merge 

In [48]:
df_gr_factor = pd.merge(
    result,
    df_dz_gr.groupby("MAF").mean().reset_index(),
    on="MAF",
)

df_gr_factor.head()

,MAF_bin,Count,Proportion,MAF,het,hom,hetxhom
0,"(0.004, 0.015]",420880,0.073546,0.01,0.494910,0.246525,0.034816
1,"(0.015, 0.025]",561792,0.098169,0.02,0.489042,0.264571,0.049269
2,"(0.025, 0.035]",370703,0.064778,0.03,0.484580,0.264148,0.060051
3,"(0.035, 0.045]",279448,0.048831,0.04,0.479331,0.272077,0.069089
4,"(0.045, 0.055]",230913,0.040350,0.05,0.473872,0.277898,0.076618


In [49]:

het_gr_exp = np.sum(df_gr_factor["Proportion"] * df_gr_factor["het"])
hom_gr_exp = np.sum(df_gr_factor["Proportion"] * df_gr_factor["hom"])
hethom_gr_exp = np.sum(df_gr_factor["Proportion"] * df_gr_factor["hetxhom"])

het_gr_exp, hom_gr_exp, hethom_gr_exp

(np.float64(0.4066165320272421),
 np.float64(0.3433085587530389),
 np.float64(0.08333073480046829))